In [1]:
# Datos: https://datosabiertos.bogota.gov.co/dataset/8c2a07f9-5948-41ca-8f7a-447753426ab9/resource/4911565b-4518-4931-93de-d117c10dbdce/download/osb_ocupacion_hosp_ucis.csv

In [2]:
import requests
import pandas as pd
from google.cloud import storage

In [3]:
URL = 'https://datosabiertos.bogota.gov.co/dataset/8c2a07f9-5948-41ca-8f7a-447753426ab9/resource/4911565b-4518-4931-93de-d117c10dbdce/download/osb_ocupacion_hosp_ucis.csv'
FILEPATH = './data/camas_bogota_raw.csv'

In [4]:
r = requests.get( URL, stream = True )

with open( FILEPATH, 'wb' ) as f:
    f.write( r.content )

In [8]:
camas = pd.read_csv( './data/camas_bogota_raw.csv', sep = ';', encoding = 'ISO-8859-1', dtype = { 'Camas Ocupadas': str, 'Camas habilitadas': str, 'Camas disponibles': str } )

In [10]:
camas[ 'Camas Ocupadas' ] = camas[ 'Camas Ocupadas' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )
camas[ 'Camas habilitadas' ] = camas[ 'Camas habilitadas' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )
camas[ 'Camas disponibles' ] = camas[ 'Camas disponibles' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )

In [11]:
camas.dtypes

IPS                        object
Servicio                   object
Grupo                      object
Camas Ocupadas              int64
Camas habilitadas           int64
Camas disponibles           int64
Porcentaje de Ocupación    object
Fecha Actualización        object
dtype: object

In [12]:
camas

,IPS,Servicio,Grupo,Camas Ocupadas,Camas habilitadas,Camas disponibles,Porcentaje de Ocupación,Fecha Actualización
0,Públicas,Hospitalización General,Neonatal,95,138,43,69%,17/10/2020
1,Públicas,Hospitalización General,Pediátrico,160,291,131,55%,17/10/2020
2,Públicas,Hospitalización General,Adulto,1007,1243,236,81%,17/10/2020
3,Públicas,Unidad de Cuidado Intermedio,Neonatal,79,126,47,63%,17/10/2020
4,Públicas,Unidad de Cuidado Intermedio,Pediátrico,4,8,4,50%,17/10/2020
5,Públicas,Unidad de Cuidado Intermedio,Adulto,22,41,19,54%,17/10/2020
6,Públicas,Unidad de cuidado intensivo,Neonatal,52,68,16,76%,17/10/2020
7,Públicas,Unidad de cuidado intensivo,Pediátrico,18,24,6,75%,17/10/2020
8,Públicas,Unidad de cuidado intensivo,Adulto,278,543,265,51%,17/10/2020
9,Privadas,Hospitalización General,Neonatal,121,149,28,81%,17/10/2020


In [13]:
camas_grouped = camas.groupby( 'Servicio' ).agg( { 'Camas Ocupadas': 'sum', 'Camas habilitadas': 'sum', 'Camas disponibles': 'sum' } ).reset_index()

In [14]:
camas_grouped[ '% Ocupación' ] = camas_grouped[ 'Camas Ocupadas' ] / camas_grouped[ 'Camas habilitadas' ]

In [15]:
camas_grouped[ 'Servicio' ] = camas_grouped[ 'Servicio' ].replace( { 'Hospitalización General': 'General', 'Unidad de Cuidado Intermedio': 'Intermedio', 'Unidad de cuidado intensivo': 'UCI' } )

In [16]:
camas_grouped.rename( columns = { '% Ocupación': 'Ocupacion' }, inplace = True )

In [17]:
ucis = pd.read_csv( 'https://storage.googleapis.com/siscovid/uci_bogota.csv' )

In [18]:
ucis_num = ucis[ 'Camas UCI ocupadas Covid-19' ].values[ -1 ]

In [19]:
ucis_num

790

In [20]:
listofzeros = [ 0 ] * 3
listofzeros[ list( camas_grouped[ 'Servicio' ].values ).index( 'UCI' ) ] = ucis_num
listofzeros

[0, 0, 790]

In [21]:
camas_grouped[ 'Camas Ocupadas COVID' ] = pd.Series( listofzeros )

In [22]:
camas_grouped[ 'Camas Ocupadas' ] = camas_grouped[ 'Camas Ocupadas' ] - camas_grouped[ 'Camas Ocupadas COVID' ]

In [23]:
camas_grouped

,Servicio,Camas Ocupadas,Camas habilitadas,Camas disponibles,Ocupacion,Camas Ocupadas COVID
0,General,6670,8439,1769,0.790378,0
1,Intermedio,507,881,374,0.575482,0
2,UCI,956,2605,859,0.670250,790


In [24]:
camas_grouped.to_csv( './data/camas_bogota.csv', index = False )

In [ ]:
#DataFrame to CSV
csv_data=camas_grouped.to_csv(index=False)

#Client, bucket and blob to acces gcloud
client= storage.Client()
bucket= client.get_bucket('siscovid')
blob=bucket.blob('camas_bogota.csv')

#Uploads the file to gcloud
blob.upload_from_string(data=csv_data, content_type='text/csv')